### Relative Affordability Calculaton

In [24]:
import pandas as pd
import numpy as np 
import geopy.distance

pd.set_option('display.max_columns', None)

In [3]:
zips_data = pd.read_csv('../nicole/clean_data/final_zip_data.csv')

In [5]:
listings_data = pd.read_csv('../nicole/clean_data/redfin_cleaned_stage_1.csv')

In [36]:
smh_data = pd.read_csv('../nicole/clean_data/SMH_Projects.csv')
smh_data['MedianSalesPrice'] = (smh_data['MinBaseSalesPrice'] + smh_data['MaxBaseSalesPrice'])/2

In [37]:
#dict of lat/long for each SMH property
smh_geo = smh_data[['MetroAreaID', 'Latitude', 'Longitude']]
smh_geo_dict = smh_geo.set_index('MetroAreaID').T.to_dict('list')

/var/folders/rq/s81db4sn0tl61l05qmsldf5r0000gn/T/ipykernel_950/2018127091.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  smh_geo_dict = smh_geo.set_index('MetroAreaID').T.to_dict('list')


In [35]:
#dict of centroid lat/long for each SMH community/metro area ID
centroids = pd.read_csv('../project/centroid_dummy .csv')
centroids_dict = centroids.set_index('MetroAreaID').T.to_dict('list')

## Inputs

#### Average Stanley Martin Base Price: 
- Average base price of all SMH homes in the selected SMH Community. 

In [39]:
#list of average sale price of each SMH Community 
smh = smh_data.groupby(['MetroAreaID']).agg({'MedianSalesPrice': ['mean']}).reset_index()
smh.columns = ['MetroAreaID', 'Average Sale Price']
smh.head()

,MetroAreaID,Average Sale Price
0,1000,700512.282609
1,1001,541461.428571
2,1003,492651.590909
3,1004,507030.000000
4,1006,342768.846154


#### Average Resale Price:
- Average resale price of the competitor houses within the radius of selected distance  from the SMH community. (Previous Year Closing price) 

In [ ]:
in_radius = []

for value in smh_geo_dict.values():

if geopy.distance.vincenty(value, coords_2).km < 

#### Average Resale Year Built: 
- Average year when the resale homes were built. (Example 1992) 

## Calculations

##### Age of Resales: 
- Average Resale Year Built — Current Year (e.g., 2021 -1992 = 29) 

##### Expected Resale Premium: 
- Based on the age of Resales, this is derived from lookup table #1. Example: For  age band of 21-30, Resale Premium is 40% 

##### Actual Resale Premium: 
- (Average Stanley Martin Base Price — Average Resale Price)/ Average Resale Price 

##### Score: 
- No score if there is no Actual Resale Premium. 
- If Actual Resale Premium > Expected Resale  Premium, then 1. 
- Else, if Actual Resale Premium< Expected Resale Premium then 3, else 2. 

##### Results: 
- Is Pricing Above or Below the Expected Premium to Resales? 
- To get this Premium to Resales value is looked up to Resale Premium Analysis table. (Table 2) Relative Affordability Ranking: 
- Based on the Score from the Calculations above. Lookup Table #3 and obtain the ranking.


In [ ]:
ideas:
map, show radius , show score 
inputs: radius length, input address (come with community), 